In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pkbar ## tracing losses during training
import pandas

In [ ]:
class Parameter():
    def __init__(self, tensor):
        self.weights = tensor
        self.gradients = np.zeros_like(self.weights)
        self.bias = np.zeros((tensor.shape[-1]))
        self.bias_gradients = np.zeros_like(self.bias)

In [ ]:
class Layer:
    def __init__(self):
        self.parameters = None
    def init_param(self, tensor):
        param = Parameter(tensor)
        self.parameters = param
        return param
    def update(self, optimizer):
        optimizer.update(self.parameters)
        

In [ ]:
class Linear(Layer):
    
    '''
    #### Objective
    A class which defines the linear layer
    '''
    
    def __init__(self, inputs, outputs):
        super().__init__()
        tensor = np.random.randn(inputs, outputs)
        self.parameters = self.init_param(tensor)
    def backward(self, D, X):
        '''
        ###### Objective
        A backward pass
        ######  Input
        partial derivative with respect to end features and start features
        ##### Output
        partial derivative with respect to start function
        '''
        return D@self.parameters.weights.T### D*theta.T
        
    def forward(self, X):
        '''
        ###### Objective
        A forward pass
        ######  Input
        start features
        ##### Output
        end features
        '''
        return X@self.parameters.weights + self.parameters.bias

In [ ]:
class Sigmoid(Layer):
    def __init__(self):
        self.parameters = None
    def backward(self, D, X):
        S = 1/(1+np.exp(-X))
        return D*(S*(1-S))
    def forward(self, X):
        return 1/(1+np.exp(-X))

In [ ]:
class Softmax(Layer):
    def __init__(self):
        self.parameters = None
    
    def backward(self, D, X):
        return D
    def softmax(self, X):
        k = np.sum(np.exp(X), axis = 1)
        t = 0
        X_out = []
        
        for i in X:
            X_out.append(list(np.exp(i)/k[t]))
            t+=1
        return np.array(X_out)
    
    def forward(self, X):
        return self.softmax(X)

In [ ]:
class SGD():
    def __init__(self, lr=0.1):
        self.lr = lr
    def update(self, param):
        '''
        ##### Objective
        Update of parameters using gradient descent
        ##### Input
        Parameters
        ##### Output
        Updated Parameters
        '''
        param.weights -= self.lr*param.gradients
        param.bias -= self.lr *param.bias_gradients

In [ ]:
class Tanh(Layer):
    def __init__(self):
        self.parameters = None
    def backward(self, D, X):
        return D*(1-(np.tanh(X)*np.tanh(X)))
    def forward(self, X):
        return np.tanh(X)

In [ ]:
class ReLU(Layer):
    def __init__(self):
        self.parameters = None
    def backward(self, D, X):
        return D*(1*(X>0))
    def forward(self, X):
        return np.maximum(0,X)

In [ ]:
class Model()
    def __init__(self):
        self.computational_graph = []
    def add(self, layer):
        '''
        ###### Objective
        Add a layer to the computational graph
        ###### Input
        layer
        
        '''
        self.computational_graph.append(layer)
    def compiler(self, loss, optimizer):
        '''
        ###### Objective 
        compile a model(give it all additional properties needed for training)
        ###### Input
        loss and optimizer to be used
        '''
        self.loss = loss
        self.optimizer = optimizer
    def forward(self, X):
        '''
        ###### Objective 
        A forward pass
        ###### Input
        Input data for model
        ###### Output
        predicted data by model, plus intermediary layers
        '''
        Y_int = X
        Y_int_list = []
        
        for layer in self.computational_graph:
            Y_int_list.append(Y_int)
            Y_ = layer.forward(Y_int)
            Y_int = Y_
        return Y_, Y_int_list
    def fit_batch(self, X,Y):
        '''
        ####### OBjective
        optimize the parameters of a particular batch
        ###### Input 
        the input and output of dataset
        
        ##### Ouput
        loss
        
        '''
        out = X
        Y_, Y_int_list = self.forward(X)
        
        D = predicted_output - target_output
        L,D = self.loss(Y_,Y)
        for Y_int, layer in zip(Y_int_list[::-1], self.computational_graph[::-1]):
            D = layer.backward(D,Y_int)
            if layer.parameters is not None:
                layer.update(self.optimizer)
        return L
    def fit(self, X,Y, epochs, bs):
        losses = []
        
        pbar = pkbar.Pbar(name='Training', target = epochs)
        kbar = pkbar.Kbar(target=epochs)
        
        for epoch in range(epochs):
            loss = 0.0
            for i in range(0, len(X), bs):
                loss += self.fit_batch(X[i:i+bs], Y[i:i+bs])
            losses.append(loss)
            kbar.update(epoch+1, values=[('loss',loss)])
        return losses

In [ ]:
X = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]], dtype = float)
Y = np.array([[0],
              [1],
              [1],
              [0]], dtype = float)

In [ ]:
EPOCHS = 10000
model = Model()
model.add(Linear(2,10))
model.add(Sigmoid())
model.add(Linear(10,1))
model.add(Sigmoid())

model.compiler(cce_loss, SGD(lr=5e-1))
losses = model.fit(X,Y, epochs = EPOCHS, bs = X.shape[0])
plt.plot(range(1,EPOCHS+1), losses)

plt.ylabel('BCE')
plt.xlabel('Number of epochs')
plt.show()
